In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import torch.nn as nn
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import math
import torchmetrics
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
batch_size=2
image_count=50
tf = ToTensor()


In [2]:

class CustomDataset(Dataset):
    def __init__(self, image_path,label):
        self.img_path=image_path
        self.label=label
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        image_tensor=self.img_path[idx]
        label_tensor=self.label[idx]
        return image_tensor,label_tensor
    
train_image_transition_path='../../data/Tiling/512/train/transition/*'
train_image_not_transition_path='../../data/Tiling/512/train/not_transition/*'
test_image_transition_path='../../data/Tiling/512/validation/transition/*'
test_image_not_transition_path='../../data/Tiling/512/validation/not_transition/*'
train_image_list=[]
train_label_list=[]
test_image_list=[]
test_label_list=[]
train_image_transition_list=glob(train_image_transition_path)
train_image_transition_label=torch.ones(len(train_image_transition_list),1)
train_image_not_transition_list=glob(train_image_not_transition_path)
train_image_not_transition_label=torch.zeros(len(train_image_not_transition_list),1)
train_image_list.extend(train_image_transition_list)
train_image_list.extend(train_image_not_transition_list)
train_label_list.extend(train_image_transition_label)
train_label_list.extend(train_image_not_transition_label)
test_image_transition_list=glob(test_image_transition_path)
test_image_transition_label=torch.ones(len(test_image_transition_list),1)
test_image_not_transition_list=glob(test_image_not_transition_path)
test_image_not_transition_label=torch.zeros(len(test_image_not_transition_list),1)
test_image_list.extend(test_image_transition_list)
test_image_list.extend(test_image_not_transition_list)
test_label_list.extend(test_image_transition_label)
test_label_list.extend(test_image_not_transition_label)

train_image_tensor=torch.empty((len(train_image_list),image_count,1,3,224,224))
for i in range(len(train_image_list)):
    train_image_file_list=glob(train_image_list[i]+'/*.jpg')
    image_index=torch.randint(low=0, high=len(train_image_file_list),size=(image_count,))
    count=0
    for index in image_index:
        image=1-tf(Image.open(train_image_file_list[index]).resize((224,224))).unsqueeze(0)
        train_image_tensor[i,count]=image.unsqueeze(0)
        count+=1

test_image_tensor=torch.empty((len(test_image_list),image_count,1,3,224,224))
for i in range(len(test_image_list)):
    test_image_file_list=glob(test_image_list[i]+'/*.jpg')
    image_index=torch.randint(low=0, high=len(test_image_file_list),size=(image_count,))
    count=0
    for index in image_index:
        image=1-tf(Image.open(test_image_file_list[index]).resize((224,224))).unsqueeze(0)
        test_image_tensor[i,count]=image.unsqueeze(0)
        count+=1
    
train_dataset=CustomDataset(train_image_tensor,train_label_list)
test_dataset=CustomDataset(test_image_tensor ,test_label_list)
dataset_size = len(test_dataset)
test_size = int(dataset_size * 0.5)
validation_size = dataset_size-test_size
validation_dataset, test_dataset = random_split(test_dataset, [validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


KeyboardInterrupt: 

In [5]:
def relu_fn(x):
    """ Swish activation function """
    return x * torch.sigmoid(x)

class Conv2dSamePadding(nn.Conv2d):
    """ 2D Convolutions like TensorFlow """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride, 0, dilation, groups, bias)

    def forward(self, x):
        ih, iw = x.size()[-2:]
        kh, kw = self.weight.size()[-2:]
        sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w//2, pad_w - pad_w//2, pad_h//2, pad_h - pad_h//2])
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
    
def drop_connect(inputs, p, training):
    """ Drop connect. """
    if not training: return inputs
    batch_size = inputs.shape[0]
    keep_prob = 1 - p
    random_tensor = keep_prob
    random_tensor += torch.rand([batch_size, 1, 1, 1], dtype=inputs.dtype)  # uniform [0,1)
    binary_tensor = torch.floor(random_tensor)
    output = inputs.to(device) / keep_prob * binary_tensor.to(device)
    return output

class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block
    """

    def __init__(self, kernel_size, stride, expand_ratio, input_filters, output_filters, se_ratio, drop_n_add):
        super().__init__()
        
        self._bn_mom = 0.1
        self._bn_eps = 1e-03
        self.has_se = (se_ratio is not None) and (0 < se_ratio <= 1)
        self.expand_ratio = expand_ratio
        self.drop_n_add = drop_n_add

        # Filter Expansion phase
        inp = input_filters  # number of input channels
        oup = input_filters * expand_ratio  # number of output channels
        if expand_ratio != 1: # add it except at first block 
            self._expand_conv = Conv2dSamePadding(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
            self._bn0 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Depthwise convolution phase
        k = kernel_size
        s = stride
        self._depthwise_conv = Conv2dSamePadding(
            in_channels=oup, out_channels=oup, groups=oup,  # groups makes it depthwise(conv filter by filter)
            kernel_size=k, stride=s, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Squeeze and Excitation layer, if desired
        if self.has_se:
            num_squeezed_channels = max(1,int(input_filters * se_ratio))  # input channel * 0.25 ex) block2 => 16 * 0.25 = 4
            self._se_reduce = Conv2dSamePadding(in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1)
            self._se_expand = Conv2dSamePadding(in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1)

        # Output phase
        final_oup = output_filters
        self._project_conv = Conv2dSamePadding(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
        self._bn2 = nn.BatchNorm2d(num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps)
    def forward(self, inputs, drop_connect_rate=0.2):
    
        # Expansion and Depthwise Convolution
        x = inputs
        if self.expand_ratio != 1:
            x = relu_fn(self._bn0(self._expand_conv(inputs)))
        x = relu_fn(self._bn1(self._depthwise_conv(x)))

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_expand(relu_fn(self._se_reduce(x_squeezed)))
            x = torch.sigmoid(x_squeezed) * x
            
        # Output phase
        x = self._bn2(self._project_conv(x))

        # Skip connection and drop connect
        if self.drop_n_add == True:
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x    
    
class EfficientNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Batch norm parameters
        bn_mom = 0.1
        bn_eps = 1e-03

        # stem
        in_channels = 3
        out_channels = 32
        self._conv_stem = Conv2dSamePadding(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Build blocks
        self._blocks = nn.ModuleList([]) # list 형태로 model 구성할 때
        # stage2 r1_k3_s11_e1_i32_o16_se0.25
        self._blocks.append(MBConvBlock(kernel_size=3, stride=1, expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, drop_n_add=False))
        # stage3 r2_k3_s22_e6_i16_o24_se0.25
        self._blocks.append(MBConvBlock(3, 2, 6, 16, 24, 0.25, False))
        self._blocks.append(MBConvBlock(3, 1, 6, 24, 24, 0.25, True))
        # stage4 r2_k5_s22_e6_i24_o40_se0.25
        self._blocks.append(MBConvBlock(5, 2, 6, 24, 40, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 40, 40, 0.25, True))
        # stage5 r3_k3_s22_e6_i40_o80_se0.25
        self._blocks.append(MBConvBlock(3, 2, 6, 40, 80, 0.25, False))
        self._blocks.append(MBConvBlock(3, 1, 6, 80, 80, 0.25, True))
        self._blocks.append(MBConvBlock(3, 1, 6, 80, 80, 0.25, True))
        # stage6 r3_k5_s11_e6_i80_o112_se0.25
        self._blocks.append(MBConvBlock(5, 1, 6, 80,  112, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 112, 112, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 112, 112, 0.25, True))
        # stage7 r4_k5_s22_e6_i112_o192_se0.25
        self._blocks.append(MBConvBlock(5, 2, 6, 112, 192, 0.25, False))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        self._blocks.append(MBConvBlock(5, 1, 6, 192, 192, 0.25, True))
        # stage8 r1_k3_s11_e6_i192_o320_se0.25
        self._blocks.append(MBConvBlock(3, 1, 6, 192, 320, 0.25, False))

        # Head 
        in_channels = 320
        out_channels = 1280
        self._conv_head = Conv2dSamePadding(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Final linear layer
        self._dropout = 0.2
        self._num_classes = 10
        self._fc = nn.Linear(out_channels, self._num_classes)
        self.fc1=nn.Linear(15680, 1000, bias=False)
    def extract_features(self, inputs):
        """ Returns output of the final convolution layer """

        # Stem
        x = relu_fn(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):          
            x = block(x)
        return x

    def forward(self, x):
        """ Calls extract_features to extract features, applies final linear layer, and returns logits. """

        # Convolution layers
        x = self.extract_features(x.to(device)).to(device)
        x = torch.flatten(x, 1).to(device)
        x=self.fc1(x).to(device)
        return x
class Custom_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn=EfficientNet().to(device)
        self.hidden_size = 128
        self.num_layers = 2
        self.rnn = nn.RNN(1000, 128, 2, batch_first=True)
        self.fc = nn.Linear(128, 1)
    def forward(self, inputs):
        total_x=torch.empty((batch_size,image_count,1000)).to(device)
        for i in range(len(inputs)):
            for j in range(len(inputs[i])):
                x=self.cnn(inputs[i,j].to(device)).to(device)
                total_x[i,j]=x.to(device)
        h0 = torch.zeros(self.num_layers,total_x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(total_x, h0)

        # 마지막 시퀀스의 출력을 사용하여 예측
        out = self.fc(out[:, -1, :]).to(device)
       
        return out
accuracy = torchmetrics.Accuracy(task="binary", num_classes=1).to(device)
model = Custom_model()
model = model.to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
summary(model,(batch_size,image_count,1,3,224,224))

Layer (type:depth-idx)                             Output Shape              Param #
Custom_model                                       [2, 1]                    --
├─EfficientNet: 1-1                                [1, 1000]                 424,970
│    └─Conv2dSamePadding: 2-1                      [1, 32, 112, 112]         864
│    └─BatchNorm2d: 2-2                            [1, 32, 112, 112]         64
│    └─ModuleList: 2-399                           --                        (recursive)
│    │    └─MBConvBlock: 3-1                       [1, 16, 112, 112]         1,448
│    │    └─MBConvBlock: 3-2                       [1, 24, 56, 56]           6,004
│    │    └─MBConvBlock: 3-3                       [1, 24, 56, 56]           10,710
│    │    └─MBConvBlock: 3-4                       [1, 40, 28, 28]           15,350
│    │    └─MBConvBlock: 3-5                       [1, 40, 28, 28]           31,290
│    │    └─MBConvBlock: 3-6                       [1, 80, 14, 14]           37,13

In [4]:
MIN_loss=5000
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
for epoch in range(1000):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    for x, y in train:
        model.train()
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = criterion(predict, y) # cost 구함
        acc=accuracy(predict, y)
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{1000} Step: {count+1} loss : {running_loss/count:.4f} accuracy: {acc_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count).cpu().detach().numpy())
#validation
    val=tqdm(validation_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            count+=1
            x=x.to(device).float()
            
            predict = model(x).to(device)
            acc=accuracy(predict, y)
            cost = criterion(predict, y)
            val_running_loss+=cost.item()
            acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{1000} Step: {count+1} loss : {val_running_loss/count:.4f}  accuracy: {acc_loss/count:.4f}")
        val_loss_list.append((val_running_loss/count))
        val_acc_list.append((acc_loss/count).cpu().detach().numpy())
    if epoch%100==1:
        plt.figure(figsize=(10,5))
        plt.subplot(1, 2, 1) 
        plt.title('loss_graph')
        plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
        plt.plot(np.arange(epoch+1),val_loss_list,label='validation_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.subplot(1, 2, 2)  
        plt.title('acc_graph')
        plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
        plt.plot(np.arange(epoch+1),val_acc_list,label='validation_acc')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.show()
        
    if MIN_loss>(val_running_loss/count):
        torch.save(model.state_dict(), '../../model/image_512/EffB0_tile150_RNN_callback.pt')
        MIN_loss=(val_running_loss/count)
torch.save(model.state_dict(), '../../model/image_512/EffB0_tile150_RNN.pt')

NameError: name 'train_dataloader' is not defined